In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.utils import Sequence, to_categorical
from keras_vggface.vggface import VGGFace
from skimage.color import gray2rgb
from skimage.transform import resize
from sklearn.svm import SVC

from load_data import LoadData


/home/mandark/.conda/envs/tf1.7/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class Restnet50Sequence(Sequence):
    def __init__(self, x_set, y_set, batch_size=16):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        return np.array([
            resize(img, (200, 200))
            for img in batch_x]), np.array(batch_y)


In [3]:
def extract_features(model, layer_name, model_name, data, mode=0, data_generator=None):
    if mode == 0:
        model.load_weights('{0}/model.h5'.format(model_name))
    elif mode == 1:
        model.load_weights('{0}/checkpoint.h5'.format(model_name))

    intermediate_layer_model = Model(inputs=model.input,
                                     outputs=model.get_layer(layer_name).output)

    if data_generator:
        print(len(data_generator))
        return intermediate_layer_model.predict_generator(data_generator, use_multiprocessing=True)
    else:
        return intermediate_layer_model.predict(data)

In [4]:


X_train, y_train = LoadData(mode=0, rgb=True).load()
X_test, y_test = LoadData(mode=2, rgb=True).load()


def load(mode=0, resize_imgs=False):
    file = ['fer2013/training.csv',
            'fer2013/publictest.csv',
            'fer2013/privatetest.csv']
    data = pd.read_csv(file[mode])

    pixels = data['pixels'].apply(
        lambda img: np.fromstring(img, sep=' '))

    X = np.vstack(pixels.values)
    X = X.astype('float32')

    X /= 255

    X = gray2rgb(X)
    X = X.reshape(-1, 48, 48, 3)

    if resize_imgs:
        X = np.array([resize(x, (200, 200)) for x in X])

    y = data['emotion'].values
    y = y.astype(np.int)
    y = to_categorical(y)

    return X, y

In [5]:

def first_model():
    vgg_model = VGGFace(include_top=False, input_shape=(48, 48, 3))
    last_layer = vgg_model.get_layer('pool5').output
    x = Flatten(name='flatten')(last_layer)
    x = Dense(4096, activation='relu', name='fc6')(x)
    x = Dense(4096, activation='relu', name='fc7')(x)
    out = Dense(7, activation='softmax', name='fc8')(x)
    model = Model(vgg_model.input, out)

    features_train = extract_features(model, 'flatten', 'vggface_vgg16v2', X_train, mode=0)
    features_test = extract_features(model, 'flatten', 'vggface_vgg16v2', X_test, mode=0)

    return features_train, features_test

In [6]:

def second_model():
    vgg_model = VGGFace(model='resnet50', include_top=False,
                        input_shape=(200, 200, 3))
    last_layer = vgg_model.get_layer('avg_pool').output
    x = Flatten(name='flatten')(last_layer)
    out = Dense(7, activation='softmax', name='classifier')(x)
    model = Model(vgg_model.input, out)

    train_generator = Restnet50Sequence(X_train, y_train, 16)
    test_generator = Restnet50Sequence(X_test, y_test, 16)

    features_train = extract_features(model, 'flatten', 'vggface_restnet50v2', X_train, mode=0,
                                      data_generator=train_generator)
    features_test = extract_features(model, 'flatten', 'vggface_restnet50v2', X_test, mode=0,
                                     data_generator=test_generator)

    return features_train, features_test

In [7]:

model1_train, model1_test = first_model()

In [8]:
model2_train, model2_test = second_model()


1795


/home/mandark/.conda/envs/tf1.7/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


/home/mandark/.conda/envs/tf1.7/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


225


/home/mandark/.conda/envs/tf1.7/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [9]:
print(model1_train.shape)
print(model1_test.shape)
print(model2_train.shape)
print(model2_test.shape)
print(np.array_equal(model2_test, model2_train))

(28709, 512)
(3589, 512)
(28709, 2048)
(3589, 2048)
False


In [10]:
print(model1_test.shape)
print(model2_test.shape)
features_train = np.concatenate([model1_train, model2_train], axis=1)
features_test = np.concatenate([model1_test, model2_test], axis=1)

(3589, 512)
(3589, 2048)


In [11]:

clf = SVC(decision_function_shape='ovo', verbose=1)
# import pdb; pdb.set_trace()
clf.fit(features_train, y_train)
score = clf.score(features_test, y_test)
print(score)


[LibSVM]

0.7183059348007802
